**Task:**
Practice training a deep neural network on the CIFAR10 image dataset:

a. Build a DNN with 20 hidden layers of 100 neurons each (that’s too many, but
it’s the point of this exercise). Use He initialization and the Swish activation
function.

b. Using Nadam optimization and early stopping, train the network on the
CIFAR10 dataset. You can load it with tf.keras.datasets.cifar10.load_
data(). The dataset is composed of 60,000 32 × 32–pixel color images (50,000
for training, 10,000 for testing) with 10 classes, so you’ll need a softmax
output layer with 10 neurons. Remember to search for the right learning rate
each time you change the model’s architecture or hyperparameters.

c. Now try adding batch normalization and compare the learning curves: is it
converging faster than before? Does it produce a better model? How does it
affect training speed?

d. Try replacing batch normalization with SELU, and make the necessary adjust‐
ments to ensure the network self-normalizes (i.e., standardize the input fea‐
tures, use LeCun normal initialization, make sure the DNN contains only a
sequence of dense layers, etc.).

e. Try regularizing the model with alpha dropout. Then, without retraining your
model, see if you can achieve better accuracy using MC dropout.

f. Retrain your model using 1cycle scheduling and see if it improves training
speed and model accuracy.

a. Building the DNN with 20 hidden layers of 100 neurons each:

In [1]:
import tensorflow as tf
from tensorflow import keras

initializer = tf.keras.initializers.HeNormal()

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100, 
                                 activation=keras.layers.Activation('swish'), 
                                 kernel_initializer=initializer))
model.add(keras.layers.Dense(10, activation="softmax"))

c:\Users\Suat\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\initializers\initializers.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


b. Training the network with Nadam optimization and early stopping:

In [2]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.cifar10.load_data()
X_train_full = X_train_full.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

model.compile(loss="sparse_categorical_crossentropy", optimizer=keras.optimizers.Nadam(lr=1e-3), metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(X_train_full, y_train_full, epochs=100, validation_split=0.2, callbacks=[early_stopping_cb])

170498071/170498071 [==============================] - 21s 0us/step


Epoch 1/100
1250/1250 [==============================] - 19s 9ms/step - loss: 2.0216 - accuracy: 0.2217 - val_loss: 1.9425 - val_accuracy: 0.2594
Epoch 2/100
1250/1250 [==============================] - 11s 9ms/step - loss: 1.8732 - accuracy: 0.2957 - val_loss: 1.8134 - val_accuracy: 0.3114
Epoch 3/100
1250/1250 [==============================] - 12s 10ms/step - loss: 1.8039 - accuracy: 0.3293 - val_loss: 1.7727 - val_accuracy: 0.3450
Epoch 4/100
1250/1250 [==============================] - 12s 10ms/step - loss: 1.7530 - accuracy: 0.3543 - val_loss: 1.7332 - val_accuracy: 0.3694
Epoch 5/100
1250/1250 [==============================] - 12s 9ms/step - loss: 1.7141 - accuracy: 0.3763 - val_loss: 1.7161 - val_accuracy: 0.3710
Epoch 6/100
1250/1250 [==============================] - 12s 9ms/step - loss: 1.6892 - accuracy: 0.3896 - val_loss: 1.6701 - val_accuracy: 0.3969
Epoch 7/100
1250/1250 [==============================] - 11s 8ms/step - loss: 1.6521 - accuracy: 0.4038 - val_loss: 1.6793

c. Adding batch normalization to the model:

In [3]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100, use_bias=False))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation('swish'))
model.add(keras.layers.Dense(10, activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy", optimizer=keras.optimizers.Nadam(lr=1e-3), metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(X_train_full, y_train_full, epochs=100, validation_split=0.2, callbacks=[early_stopping_cb])

Epoch 1/100
1250/1250 [==============================] - 27s 13ms/step - loss: 1.9716 - accuracy: 0.2736 - val_loss: 2.0564 - val_accuracy: 0.2859
Epoch 2/100
1250/1250 [==============================] - 15s 12ms/step - loss: 1.7620 - accuracy: 0.3653 - val_loss: 1.8347 - val_accuracy: 0.3569
Epoch 3/100
1250/1250 [==============================] - 16s 13ms/step - loss: 1.6748 - accuracy: 0.4020 - val_loss: 1.8056 - val_accuracy: 0.3785
Epoch 4/100
1250/1250 [==============================] - 15s 12ms/step - loss: 1.6040 - accuracy: 0.4279 - val_loss: 1.7163 - val_accuracy: 0.3933
Epoch 5/100
1250/1250 [==============================] - 15s 12ms/step - loss: 1.5571 - accuracy: 0.4445 - val_loss: 1.6618 - val_accuracy: 0.4163
Epoch 6/100
1250/1250 [==============================] - 16s 13ms/step - loss: 1.5110 - accuracy: 0.4629 - val_loss: 1.7899 - val_accuracy: 0.3813
Epoch 7/100
1250/1250 [==============================] - 17s 13ms/step - loss: 1.4652 - accuracy: 0.4792 - val_loss: 1

d. Replacing batch normalization with SELU:

In [4]:
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.initializers import lecun_normal
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Activation, Dropout

def selu(x):
    alpha = 1.67326
    scale = 1.0507
    return scale * K.elu(x, alpha)

model = Sequential()
model.add(Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(Dense(100, activation=selu, kernel_initializer=lecun_normal()))
model.add(Dense(10, activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy", optimizer=Nadam(lr=1e-3), metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(X_train_full, y_train_full, epochs=100, validation_split=0.2, callbacks=[early_stopping_cb])

Epoch 1/100
1250/1250 [==============================] - 21s 11ms/step - loss: 2.1147 - accuracy: 0.1889 - val_loss: 1.9534 - val_accuracy: 0.2606
Epoch 2/100
1250/1250 [==============================] - 14s 11ms/step - loss: 1.9518 - accuracy: 0.2628 - val_loss: 1.9829 - val_accuracy: 0.2902
Epoch 3/100
1250/1250 [==============================] - 13s 10ms/step - loss: 1.9128 - accuracy: 0.2810 - val_loss: 1.8798 - val_accuracy: 0.3027
Epoch 4/100
1250/1250 [==============================] - 12s 10ms/step - loss: 1.8576 - accuracy: 0.3065 - val_loss: 1.8424 - val_accuracy: 0.3208
Epoch 5/100
1250/1250 [==============================] - 13s 10ms/step - loss: 1.8347 - accuracy: 0.3182 - val_loss: 1.8773 - val_accuracy: 0.3111
Epoch 6/100
1250/1250 [==============================] - 14s 11ms/step - loss: 1.8118 - accuracy: 0.3258 - val_loss: 1.8038 - val_accuracy: 0.3393
Epoch 7/100
1250/1250 [==============================] - 14s 11ms/step - loss: 1.8097 - accuracy: 0.3318 - val_loss: 1

e. regularizing the model with alpha dropout

In [8]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100, activation=keras.layers.Activation('swish'), kernel_initializer=initializer))
    model.add(keras.layers.AlphaDropout(rate=0.1))
model.add(keras.layers.Dense(10, activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy", optimizer=keras.optimizers.Nadam(lr=1e-3), metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(X_train_full, y_train_full, epochs=100, validation_split=0.2, callbacks=[early_stopping_cb])

Epoch 1/100
1250/1250 [==============================] - 20s 11ms/step - loss: 2.3703 - accuracy: 0.1175 - val_loss: 4.1109 - val_accuracy: 0.1696
Epoch 2/100
1250/1250 [==============================] - 13s 10ms/step - loss: 2.1535 - accuracy: 0.1595 - val_loss: 10.8815 - val_accuracy: 0.1739
Epoch 3/100
1250/1250 [==============================] - 13s 10ms/step - loss: 2.1115 - accuracy: 0.1663 - val_loss: 6.5563 - val_accuracy: 0.1711
Epoch 4/100
1250/1250 [==============================] - 13s 10ms/step - loss: 2.0836 - accuracy: 0.1764 - val_loss: 9.1327 - val_accuracy: 0.1697
Epoch 5/100
1250/1250 [==============================] - 13s 10ms/step - loss: 2.0640 - accuracy: 0.1826 - val_loss: 7.6539 - val_accuracy: 0.1666
Epoch 6/100
1250/1250 [==============================] - 13s 11ms/step - loss: 2.0536 - accuracy: 0.1777 - val_loss: 7.9135 - val_accuracy: 0.1751
Epoch 7/100
1250/1250 [==============================] - 13s 10ms/step - loss: 2.0444 - accuracy: 0.1782 - val_loss: 

In [9]:
import numpy as np

# make predictions with MC dropout
y_probas = np.stack([model.predict(X_test, batch_size=128, verbose=1)
                     for sample in range(100)])

# compute mean and standard deviation of predictions
y_mean = y_probas.mean(axis=0)
y_std = y_probas.std(axis=0)

# evaluate accuracy
accuracy = np.mean(np.equal(y_test, np.argmax(y_mean, axis=1)))
print("Test accuracy:", accuracy)

79/79 [==============================] - 0s 4ms/step
Test accuracy: 0.1


f. Retraining model using 1cycle scheduling

In [41]:
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras import backend as K

def one_cycle_lr(epoch, lr):
    max_lr = 0.05
    end_percentage = 0.1
    step_size = 5 * (len(X_train_full) // 128)
    midpoint = step_size // 2
    momentum = 0.95
    
    if epoch <= midpoint:
        new_lr = ((max_lr / end_percentage) / 2) * epoch * end_percentage
    else:
        new_lr = ((max_lr / end_percentage) / 2) * (1 - (epoch - midpoint) * (1 - end_percentage) / (step_size - midpoint))
        
    model.optimizer.lr = new_lr
    model.optimizer.beta_1 = momentum
    return new_lr


lr_scheduler = LearningRateScheduler(one_cycle_lr)

model.compile(loss="sparse_categorical_crossentropy", optimizer=keras.optimizers.Nadam(lr=1e-3, beta_1=0.95), metrics=["accuracy"])
history = model.fit(X_train_full, y_train_full, epochs=20, validation_split=0.2, batch_size=128, callbacks=[lr_scheduler])


Epoch 1/20
313/313 [==============================] - 16s 27ms/step - loss: nan - accuracy: 0.0997 - val_loss: nan - val_accuracy: 0.1014 - lr: 0.0000e+00
Epoch 2/20
313/313 [==============================] - 8s 25ms/step - loss: nan - accuracy: 0.0997 - val_loss: nan - val_accuracy: 0.1014 - lr: 0.0250
Epoch 3/20
313/313 [==============================] - 8s 25ms/step - loss: nan - accuracy: 0.0997 - val_loss: nan - val_accuracy: 0.1014 - lr: 0.0500
Epoch 4/20
313/313 [==============================] - 8s 26ms/step - loss: nan - accuracy: 0.0997 - val_loss: nan - val_accuracy: 0.1014 - lr: 0.0750
Epoch 5/20
313/313 [==============================] - 8s 26ms/step - loss: nan - accuracy: 0.0997 - val_loss: nan - val_accuracy: 0.1014 - lr: 0.1000
Epoch 6/20
313/313 [==============================] - 8s 25ms/step - loss: nan - accuracy: 0.0997 - val_loss: nan - val_accuracy: 0.1014 - lr: 0.1250
Epoch 7/20
313/313 [==============================] - 8s 25ms/step - loss: nan - accuracy: 0.09